## **Sentiment analysis using TF-IDF embeddings and LSTM.**




In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import zipfile

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import keras

from keras.models import Model
from keras.layers import Dense, Activation, concatenate, Embedding, Input

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

Loading the Dataset.

In [44]:
df = pd.read_json('/content/Amazon_Instant_Video_5.json', lines = True)
df.head()

reviewerID        asin  ... unixReviewTime   reviewTime
0  A11N155CW1UV02  B000H00VBQ  ...     1399075200   05 3, 2014
1  A3BC8O2KCL29V2  B000H00VBQ  ...     1346630400   09 3, 2012
2   A60D5HQFOTSOM  B000H00VBQ  ...     1381881600  10 16, 2013
3  A1RJPIGRSNX4PW  B000H00VBQ  ...     1383091200  10 30, 2013
4  A16XRPF40679KG  B000H00VBQ  ...     1234310400  02 11, 2009

[5 rows x 9 columns]

In [45]:
len(df)

37126

In [46]:
df = df.sample(n=10000)

In [47]:
len(df)

10000

In [48]:
df["textlength"] = df["reviewText"].apply(len)

In [49]:
df.describe()

overall  unixReviewTime    textlength
count  10000.000000    1.000000e+04  10000.000000
mean       4.210700    1.376310e+09    523.692500
std        1.128465    3.125918e+07    856.556253
min        1.000000    1.155686e+09      6.000000
25%        4.000000    1.367626e+09    146.000000
50%        5.000000    1.384819e+09    234.500000
75%        5.000000    1.394086e+09    486.000000
max        5.000000    1.406074e+09  13766.000000

In [50]:
df.head()

reviewerID        asin  ...   reviewTime textlength
33306  A3RYSWLDLZ91DB  B00I3MMN4I  ...  02 11, 2014        414
26176   AWH5ZZ8UXY1QJ  B00D844FUQ  ...  09 17, 2013        455
4393   A317L56T9Q55C2  B002TNS012  ...   05 6, 2011        337
17777  A3KJ5PP49UTGB8  B009HNCUYK  ...  01 31, 2014        145
28728   AG55C6IERCDAZ  B00ESNKDHA  ...  10 13, 2013       1394

[5 rows x 10 columns]

In [51]:
df.tail()

reviewerID        asin  ...   reviewTime textlength
2673   A196QGK8G76PQF  B001GRVMLM  ...  04 23, 2014        383
30123  A1O2TG3FAE733Y  B00F3WOO70  ...  12 16, 2013        113
30241   AKD0Y0W8NSK0D  B00F406S2U  ...  07 14, 2014        294
13720  A3ALJT9CJBMZWS  B007UMK3SA  ...  06 10, 2014        134
26850   AHWMZQKU3FTI1  B00DCLZMQG  ...  01 28, 2014       2047

[5 rows x 10 columns]

In [52]:
df['reviewText']

33306    We can only hope Amazon picks up this series a...
26176    Like the first VHS, this one delivers some of ...
4393     Once I started watching "V" The First Season i...
17777    I have loved this show from the beginning,but ...
28728    My wife and I really look forward to each new ...
                               ...                        
2673     each episode is unique in itself and you want ...
30123    The series always has something different and ...
30241    Another show that is very well done. It can be...
13720    Same story line. Interesting cultural shift an...
26850    I wish Hollywood would Grow UP! Similar to mos...
Name: reviewText, Length: 10000, dtype: object

In [53]:
df = df[['reviewText', 'overall']]
df.head()

reviewText  overall
33306  We can only hope Amazon picks up this series a...        5
26176  Like the first VHS, this one delivers some of ...        3
4393   Once I started watching "V" The First Season i...        5
17777  I have loved this show from the beginning,but ...        3
28728  My wife and I really look forward to each new ...        5

In [54]:
df['overall'] = [1 if overall > 2 else 0 
                            for overall in df['overall'].values]
df.head()

reviewText  overall
33306  We can only hope Amazon picks up this series a...        1
26176  Like the first VHS, this one delivers some of ...        1
4393   Once I started watching "V" The First Season i...        1
17777  I have loved this show from the beginning,but ...        1
28728  My wife and I really look forward to each new ...        1

In [55]:
df.shape

(10000, 2)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 33306 to 26850
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  10000 non-null  object
 1   overall     10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 234.4+ KB


In [57]:
df['overall'].value_counts()

1    9004
0     996
Name: overall, dtype: int64

Spliting the data into train and test sets:

In [58]:
X = df['reviewText']
y = df['overall']

X_train_sentences, X_test_sentences, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

**Tokenizing the words.**

In [59]:
vocab_size = 5000
embedding_dim = 16
max_len = 100
trunction_type='post'
padding_type='post'
oov_token = "<OOV>"

In [60]:
vectorizer = TfidfVectorizer(max_features=300)
vectorizer = vectorizer.fit(X_train_sentences)
df_train = vectorizer.transform(X_train_sentences)
tokenize = Tokenizer()
tokenize.fit_on_texts(X_train_sentences)

In [61]:
word_index = tokenize.word_index

`texts_to_sequences` convert the words into sequences so that a complete sequence of numbers can represent every sentence. 

In [71]:
X_train = tokenize.texts_to_sequences(X_train_sentences)
X_test = pad_sequences(X_train, maxlen=max_len)

In [72]:
X_train[9:11]

[[1,
  74,
  35,
  50,
  1359,
  374,
  5,
  829,
  217,
  1,
  1779,
  789,
  3,
  2304,
  3153,
  1,
  2130,
  5,
  1,
  1016,
  2001,
  6,
  1029,
  91,
  6,
  1,
  279,
  3981,
  5,
  1,
  74],
 [13,
  29,
  178,
  118,
  863,
  1,
  1893,
  1267,
  61,
  602,
  48,
  1,
  279,
  93,
  2,
  3,
  170,
  335,
  165,
  10,
  6077,
  3154,
  14,
  50,
  335,
  10,
  1,
  31,
  210,
  804,
  210,
  1607,
  210,
  198,
  107,
  16]]

Using `pad_sequences`function to make it uniform.

In [73]:
X_train_pad = pad_sequences(X_train,maxlen=max_len, padding=padding_type, 
                            truncating=trunction_type)

X_test_pad = pad_sequences(X_test,maxlen=max_len, padding=padding_type, 
                           truncating=trunction_type)

### **TF-IDF Embedding**


In [74]:
from keras.layers import Dense, Activation, concatenate, Embedding, Input
input_tfidf = Input(shape=(embedding_dim ,))
input_text = Input(shape=(vocab_size,))

Finally we create the embedding layer:


In [76]:
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_len)(input_text)                           

### **Build the model**

In [104]:
mean_embedding = keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=1))(embedding_layer)
#concatenated = concatenate([input_tfidf, mean_embedding])
dense1 = Dense(256, activation='relu')(mean_embedding)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(8, activation='sigmoid')(dense2)
model = Model(inputs=[input_tfidf, input_text], outputs=dense3)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 5000)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 5000, 16)     80000       input_6[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 16)           0           embedding_2[0][0]                
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 256)          4352        lambda_7[0][0]                   
____________________________________________________________________________________________

In [102]:
callbacks = EarlyStopping(monitor = 'val_accuracy', 
                          mode = 'max', 
                          patience = 10,
                          verbose = 1)

Train the model for 5 epochs.

In [ ]:
history = model.fit(X_train_pad, y_train,
                    batch_size=32,
                    epochs=5,
                    validation_data=(X_test_pad, y_test),
                    verbose=1,
                    callbacks=callbacks
                    )

Predictions on a test set:

In [ ]:
loss, accuracy = model.evaluate(X_test_pad,y_test)
print('Test accuracy :', accuracy)

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
model.save("Tf-Idf_model")

Adding the dropout layer for decrease overfitting and  we increase number of epochs to 10.

In [ ]:
model2 = Sequential([
    embedding_layer,
    Bidirectional(LSTM(64, return_sequences=True)), 
    Bidirectional(LSTM(32)),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-4)
model2.compile(loss='binary_crossentropy', 
               optimizer=optimizer,
               metrics=['accuracy'])

model2.summary()

In [ ]:
callbacks = EarlyStopping(monitor = 'val_accuracy', 
                          mode = 'max', 
                          patience = 10,
                          verbose = 1)

In [ ]:
history2 = model2.fit(X_train_pad, y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(X_test_pad, y_test),
                    verbose=1,
                    callbacks=callbacks)

Predictions:

In [ ]:
loss, accuracy = model2.evaluate(X_test_pad,y_test)
print('Test accuracy :', accuracy)

In [ ]:
plot_graphs(history2, "accuracy")
plot_graphs(history2, "loss")